本文学习于论文《Ask Me Anything: Dynamic Memory Networks　for Natural Language Processing》，作者:Ankit Kumar.

# 论文解读

## 摘要: Abstract

自然语言处理中的大多数任务都可以转换为**在语言输入上的问答(QA)问题**。我们引入了**动态记忆网络ＤＭＮ**，一个统一的神经网络框架，它可以处理**输入序列(input sequences)**和**问题(questions)**,形成**语义(semantic)**和**情景记忆(episodic memories)**，生成**相关回答(relevant answers)**。**问题**触发了一个**迭代注意过程**,该过程允许模型**将其注意力放在先前迭代的结果上**,然后在**分层循环序列模型**上推导这些结果来产生**答案**。**DMN模型**可以端到端地进行训练，并获得在几种类型的任务和数据集上的最新结果:问答(Facebook’s bAbI dataset),词性标注的序列建模(WSJ-PTB),指代消岐(Quizbowl dataset),用于情感分析的文本分类 (Stanford Sentiment　Treebank)。**DMN**仅仅依赖于训练好的**词向量表示**，并且不需要字符串匹配或人工特征。

## 引言: Introduction

问答(QA)是一个复杂的自然语言处理任务，它要求理解文本的含义，并且有能力基于相关事实进行推理。大多数自然语言处理任务都可以视为一个问答问题:高层次的任务，如机器翻译;序列建模任务，如命名实体识别ＮＥＲ或词性标注ＰＯＳ;文本分类，如情感分析;甚至多句联合分类问题，如指代消岐。

## 动态记忆网络: The Dynamic Memory Network

ＤＭＮ是用于在输入上询问问题的通用建模框架。ＤＭＮ的目标，首先便是计算输入的向量表示;然后给定一个问题，生成正确的答案。它包含以下几个模块:

### 输入模块: Input Module

输入模块计算输入的一个有用的表示，这样相关的事实可以在之后被检索。通常，输入模块可以被视作是计算返回最终的向量表示的函数的中间步骤。输入模块将这些中间值传给情景记忆模块(episodic memory module),它会以问题为条件通过其注意力机制完成计算。我们将输入模块提供给情景记忆的表示，视为通过在情景记忆上所推理的事实。

![DMN_EXAMPLE.png](./image/DMN_EXAMPLE.png)

在自然语言处理中，我们有一个序列，它有$T_{I}$个单词，如下:$w_{1}^{I},...,w_{T_{I}}^{I}$。输入模块计算循环序列模型的隐藏状态。

我们使用$Glove$词向量来捕获上下文独立的表示(context-independent representations),并用这些词嵌入值来初始化**ＤＭＮ的嵌入层**。词嵌入作为循环网络的输入用来计算隐藏的事实状态(hidden fact states):

$$c_{t} = SEQ\_MODEL\left(L\left[w_{t}^{I}\right],h_{t-1}\right)$$

这里$L$是词嵌入矩阵，$w_{t}^{I}$是输入序列中第$t$个单词。特别地，我们使用门循环神经网络ＧＲＵ。

如果我们对循环神经网络进行子采样(subsample),则输入模块将仅返回与原始故事中的句末标记(end-of-sentence markers)相对应的隐藏状态$c_{t}$。除此以外，输入模块返回所有单词的隐藏状态$c_{t}$。为了方便标记，在将来的模块中，我们将这些状态中的任何一个称为$c_{t}$,并在实验部分中说明是否使用了子采样。

### 语义记忆模块: Semantic Memory Module

**语义记忆模块**包含了两部分:**存储单词概念**和**关于它们的事实**。我们用$Glove$词向量来初始化嵌入矩阵(embeddings)。该模块可以包括gazeteers或其他形式的显性知识库，但在这里我们不使用它们。

### 问题模块: Question Module

该模块将一个问题(question)映射到一个表示，这个表示能够被用于从输入模块中查询特殊事实。我们的问题是包含了$T_{Q}$个单词的$w_{t}^{Q}$序列。我们通过

$$q_{t} = GRU\left(L\left[w_{t}^{Q},q_{t-1}\right]\right)$$

来计算每个隐藏状态，这里的ＧＲＵ和嵌入权重，与输入模块是共享的。最终的问题向量被定义为:$q = q_{T_{Q}}$。

### 情景记忆模块: Episodic Memory Module

情景记忆模块(episodic memory module)以问题为条件，从输入模块中检索事实。然后，它在这些事实上进行推理，以产生**最终的表示(final representation)**，而问题模块会利用这个最终的表示来生成一个回答(answer)。我们将这个**最终的表示**视为一个**记忆(memory)**。重要的是，我们允许模块对事实进行多次传递(take multiple passes)，将注意力集中在每次传递的不同事实上。每次传递会产生一个情景片段(episode),接着这些情景片段被聚合到记忆中。赋予我们的模块这种片段组件允许其注意力机制更有选择地参与(attend)每一次传递的特定事实,因为它可以在稍后的传递中处理其他更重要的事实。它还允许一种过渡推理，因为第一遍可能会发现需要检索其他事实。

例如，在下图中，我们被问道"**Where is the football?**"，在第一次迭代中，模型应该处理第七句("**John put down the football.**"),因为问题询问了"**football**"。只有当模型看到"**John**"是相关的，它才能推断出第二次迭代应该检索**John**所在的位置。在这个例子中，取自Facebook的bAbi任务的真实测试问题，确实可以看到这种行为。请注意，第二次迭代在句子２中错误地设置了一些权重，这有点直观，因为句子２是**John**出现的另一个地方。

![DMN_MODEL.png](./image/DMN_MODEL.png)

在其一般形式中，情景记忆模块的特征在于注意力机制，它是在给定注意力机制的输出和来自输入模块的事实的情况下返回一个情景的功能，以及将情景聚合为记忆的功能。

在本论文中，我们使用门函数(gating function)作为我们的注意力机制。它将如下内容(即:对于每次传递$i$,一个候选事实$c_{t}$,一个先前状态$m^{i-1}$和问题$q$)作为输入来计算门:

$$g_{t}^{i} = G(c_{t},m^{i-1},q)$$

状态通过如下的ＧＲＵ来更新:

$$m^{i} = GRU(e^{i},m^{i-1})$$

这里$e^{i}$是在第$i$次传递(DMN会反复阅读ｉｎｐｕｔ，第几次传递表示第几次阅读，例如episode memory module中两层$e^{i}$，其中$i$就表示第几次阅读)时，被计算出的情景片段(episode)。

这个状态可以随机初始化，但是在实践中，我们发现将其初始化为问题的向量表示本身也有帮助；例如，$m^{0} = q$。函数$Ｇ$返回单个标量，并被定义为如下:

$$z(c,m,q) = \left[c,m,q,c\circ q,c\circ m,|c-q|,|c-m|,c^{T}W^{(b)}q,c^{T}W^{(b)}m \right],(1)$$

$$G(c,m,q) = \sigma\left(W^{(2)}tanh\left(W^{(1)}z(c,m,q) + b^{(1)}\right) + b^{(2)}\right),(2)$$

为了计算第$i$次的传递的情景片段(episode),我们在$T_{C}$个事实$c_{t}$的序列上使用修订过的ＧＲＵ，赋予我们的**门**。情景(episode)是ＧＲＵ的最后状态:

$$h_{t}^{i} = g_{t}^{i}GRU\left(c_{t},h_{t-1}^{i}\right) + (1-g_{t}^{i})h_{t-1}^{i},e^{i} = h_{T_{C}}^{i},(3)$$

最后，为了将第$T_{p}$个情景$e^{i}$聚合到记忆(memory)中,我们使用同样的ＧＲＵ来更新注意力机制的状态:

$$m^{i} = GRU(e^{i},m^{i-1})$$

同时我们用$m = m^{T_{P}}$来设置记忆$m$。这相当于简单地将记忆设置为注意力机制的最终状态，但是我们在此将其描述为它自己的计算，以突出显示这些子组件的潜在模块性。

对于标记那些事实对于给定问题是重要的数据集，例如Facebook的bAbi数据集，等式(2)的门可以使用标准交叉熵分类误差函数来监督训练。我们还为事实添加了一个特殊的传递结束表示($<EOS>$),如果门函数选择了这种表示，则停止迭代注意过程。否则，对于没有明确监督的数据集，我们设置最大传递次数。整个模块是端到端可微分的。

#### 序列建模模块: Sequence Modeling

可直接将**ＤＭＮ**应用于序列建模。在序列建模任务中，我们期望标记原始序列中的每一个单词。因此，我们希望每一个单词有一个向量表示。为此，我们以与上面相同的方式运行ＤＭＮ，每个单词一次。当为单词$t$运行ＤＭＮ时，我们用$e^{i} = h_{t}^{i}$来代替等式(3)，而不是为第$i$次传递设置情景为修订ＧＲＵ的最终状态。注意，第一次传递的门对于每个单词是相同的，就像问题是相同的。这使得仅需计算这些门一次来加速实现。然而，后续传递的门是不同的，因为情景不同。

情景记忆模块的最后输出是记忆$m$,它进入回答模块(answer module)。在序列建模中，每个单词独特的记忆$m$会独立地传送给回答模块。

### 回答序列: Answer Sequence

回答序列模块将记忆(memory)解码成一个表示答案的单词序列。

我们使用ＧＲＵ，并且将初始的隐藏状态设置为记忆$a_{0} = m$。随后的隐藏状态将最后隐藏状态和先前预测的输出$y_{t-1}$以及问题作为输入:

$$a_{t} = GRU\left(\left[y_{t-1},q\right],a_{t-1}\right),y_{t} = softmax(W^{(a)}a_{t}),(4)$$

$W^{(a)}$是标准的ｓｏｆｔｍａｘ层。输出用添加有特殊序列结束标记(\<EOS\>)的正确序列的交叉熵误差分类损失函数来训练。在测试时，我们不停生成单词直到生成序列结束标记。

### 训练: Training

训练被视为一个监督分类问题，以最小化答案序列的交叉熵误差。对于具有门监督的数据集，例如bAbi，我们还将门的交叉熵误差包括咋总损失中。

### 实验: Experiments

ＤＭＮ学习算法在问答(babi)、词性标注、情感分析和指代消岐上取得了最好的结果。

## 结论: Conclusion

我们认为ＤＭＮ对于许多ＮＬＰ任务是一个潜在的通用模型。整个模型可以通过一个虽然复杂的目标函数进行端到端的训练。该模型使用来自神经科学的一些想法，例如已知为复杂类型的推理所需的语义和情景记忆。

# 深入理解

本文学习于[CS224n笔记16 DMN与问答系统](http://www.hankcs.com/nlp/cs224n-dmn-question-answering.html)

DMN将所有NLP任务视作QA问题。**模仿人类粗读文章和问题，再带着问题反复阅读文章的行为**,利用DMN这个通用框架漂亮地解决了从词性标注、情感分析到机器翻译、QA等一系列任务。

## 是否所有NLP任务都可视作QA？

问答系统举例如下:
![DMN_EXAMPLE.png](./image/DMN_EXAMPLE.png)

在old-school NLP系统中，必须手工整理一个“知识库”；然后在这个知识库上做规则推断。DMN完全不同于这种小作坊，它能够直接从问答语料中学习所有必要的知识表达。DMN还可以在问答中做情感分析、词性标注和机器翻译,所以构建一个joint model用于通用QA成为终极目标。要实现这个目标，有两个障碍:

### 前无古人

没有任何已有研究，探讨过如何让单个模型学会这么多的任务。每种任务都有独特的特点，适合不同的神经网络来做:
![DMN_3.jpg](./image/DMN_3.jpg)

### 全才难得

Fully joint multitask learning（同一个decoder/classifier，不仅仅共享词向量，而应该共享全部参数）非常困难。有些不成功的研究发现，只能在低层（词向量）共享参数、如果任务之间没有直接关联则会顾此失彼。

## 动态记忆网络: Dynamic Memory Networks

DMN仅仅解决了第一个问题，虽然有些超参数还是得因任务而异，但总算是个通用的架构了。

## 回答问题的启发

假设有个阅读理解题目:
![DMN_QUESTION.jpg](./image/DMN_QUESTION.jpg)

你无法记住全文，但看了问题之后，只要带着问题扫几眼原文，你就能找出答案，这种现象启发了DMN。

## DMN结构

![DMN_MODEL.png](./image/DMN_MODEL.png)

左边$Semantic Memory Module$输入每个句子中每个单词的词向量，送入**Input Module**的**GRU**中。同样对于**Question Module**,其结构也是一个**ＧＲＵ**,两个**ＧＲＵ**可以共享权值。

**Question Module**计算出一个**Question Vector q**,根据**q**应用**Attention**机制，以回顾**Input Module**中**input**的不同时刻。根据**Attention**强度的不同，忽略了一些**input**,而注意到另一些**input**。这些**input**进入**Episodic Memory Module**,注意到图片中问题是关于**足球位置**的，那么所有与足球及位置相关的**input**被送入该模块。

**Episodic Memory Module**的结构同样也是一个**ＧＲＵ**，我们将其最后迭代次数的**episodic memory representation**的$m^{i}$输入$Answer Module$中，利用$softmax$得到答案序列。

有人问ＤＭＮ能否学会新类型的问题，答案是否定的。图中**Episodic Memory Module**中有两条线，分别代表着问题$q$第一次阅读**Input Module**的输出$c_{t}$而产生的**episode representation**$e_{t}^{1}$,以及带着问题**q**第二次阅读**Input Module**的输出$c_{t}$而产生的**episode representation**$e_{t}^{２}$,这两个会分别通过**ＧＲＵ**传递得到**episodic memory representation**$m^{1}$和$m^{2}$。

### Input Module

![DMN_input.jpg](./image/DMN_input.jpg)

输入模块**Input Module**接受$T_{I}$个输入单词，输出$T_{C}$个"事实(facts)"的表示。如果输出是一系列词语，那么有$T_{C} = T_{I}$;如果输出是一系列句子，那么约定$T_{C}$表示句子的数量，$T_{I}$表示句子中单词的数量。我们使用简单的**GRU**读入句子，得到**句子中每个时刻的隐藏状态**

$$h_{t} = GRU(x_{t},h_{t-1})$$

其中，$x_{t} = L\left[w_{t}\right]$,表示单词的词向量，**$L$**是**Embedding matrix**,$w_{t}$是时刻**t**的单词。我们用$c_{t}$表示每个句子最后时刻输出的隐藏状态，并称其为**事实fact**。

事实上，**Input Module**中的单向**GRU**可以升级为**Bi-GRU**:
![DMN_input_BI_GRU.jpg](./image/DMN_input_BI_GRU.jpg)

句子序列中，**Bi-GRU**的最后时刻输出，包括正反两个方向的$c_{t}$，将它们拼接在一起，用来表示双向隐藏状态，并称其为**事实fact**。

### Question Module

![DMN_QUESTION_MODULE.jpg](./image/DMN_QUESTION_MODULE.jpg)

与**Input Module**一样，用一个标准的**GRU**读取问题单词序列，并用相同的**Embedding matrix L**,得到问题的**单词序列最后一个隐藏状态**

$$q_{t} = GRU(L\left[w_{t}^{Q}\right],q_{t-1})$$

### Episodic Memory Module

![DMN_Episodic Memory.jpg](./image/DMNEpisodicMemory.jpg)

DMN与其他网络最大的不同之处在于:它会多次阅读输入句子，每次只注意句子的**fact表示**的一个子集。

输入模块**Input Module**的**Bi-GRU**产生的**fact表示**，**Episodic Memory**模块也使用**Bi-GRU**产生**Episodic Memory representation**,并用注意力机制(Attention Mechanism)为每个句子产生一个**episodic representation**$e_{t}^{i}$,其中**t**表示输入模块中的第 t 个句子，**i**表示第 i 次迭代(pass)。

记第 i 个pass产生的**Episodic Memory representation**为$m^{i}$,

其中，**Episodic Memory representation**初始化为$m^{0} = q$,即用问题模块的输出$q_{t}$来初始化。后续的$m^{i}$通过ＧＲＵ进行处理，以如下公式进行更新:

$$m^{i} = GRU(e^{i},m^{i-1})$$

而**episodic representation**使用**Input Module**的隐藏状态进行更新:

$$h_{t}^{i} = g_{t}^{i}GRU\left(c_{t},h_{t-1}^{i}\right) + (1-g_{t}^{i})h_{t-1}^{i}$$

$$e^{i} = h_{T_{C}}^{i}$$

其中,**g**是**Attention mechanism**,有多种计算方式。请注意，这里的$h_{t}^{i}$与输入模块中的$h_{t}$不是一回事，因为这里的输入不在是单词的向量$x_{t}$，而是每个句子最后时刻的输出隐藏状态$c_{t}$。同时，我们应该注意到，在每次迭代时，由于注意力机制，$e_{t}^{i}$都会发生变化。

在最初的ＤＭＮｐａｐｅｒ中，作者验证了下列公式是最好的:

$$g_{t}^{i} = G(c_{t},m^{i-1},q)$$

$$z(c,m,q) = \left[c,m,q,c\circ q,c\circ m,|c-q|,|c-m|,c^{T}W^{(b)}q,c^{T}W^{(b)}m \right]$$

$$G(c,m,q) = \sigma\left(W^{(2)}tanh\left(W^{(1)}z(c,m,q) + b^{(1)}\right) + b^{(2)}\right)$$

这里最重要的是**z**的计算，通过句子向量、问题向量、上一个记忆向量之间原始形式与element-wise乘积(相似性)的按行拼接，得到一个超长的列向量。然后非线性激活传播一层，**softmax**得到**g**,表示每次阅读对每个句子的关注程度。

这里两条线代表两个GRU，即阅读两遍。这也是个超参数。Richard说最开始多个pass的Memory也通过GRU合并，后来发现只留两个pass直接视作两层神经网络也能解决问题，能简单就不要复杂。

### Answer Module

**Answer Module**就是一个简单的ＧＲＵ Decoder,接受上次输出的单词(是一个one-hot向量)，以及**Episodic Memory**,输出一个单词:

$$ a_{t} = GRU\left(\left[y_{t-1},q\right],a_{t-1}\right)$$

$$ y_{t} = softmax(W^{(a)}a_{t})$$